In [1]:

from gibbs.dataclass import GibbsResult,get_results
import numpy as np
from gibbs.utils import simple_purify_hamiltonian,printarray,spectral_dec,candidate,candidateV2,candidateV3, lattice_hamiltonian, noise_hamiltonian
import matplotlib.pyplot as plt
from gibbs.learning.bayesian_learning_v2 import BayesianLearning
from gibbs.learning.constraint_matrix import ConstraintMatrixFactory
from gibbs.learning.klocal_pauli_basis import KLocalPauliBasis
from gibbs.plots.plots import compare_preparations, axis_locality,preparation
import plotly.express as px
from qiskit.quantum_info import state_fidelity,SparsePauliOp

%load_ext autoreload
%autoreload 2

Okay, we have a lot of results. It seems like Bayesian Learning per se will not work. Unless we have very very good VarQITE the prepared states will be too different from one iteration to the next. That leaves us with naive constraint matrix reconstructions. We can speak about why the error is not that big when we consider that other methods in litterature oversample quite a lot. We can probably do a factor or two times more sampling with the naive approach, which should lead to a much better reconstruction.

The main problem with that approach is that we won't know the temperature of our preparation. Let's prepare some hamiltonians and see how the fidelity evolves with error in temperature.

In [2]:
H = lattice_hamiltonian(4,-1,1/4,["Y"],["XX","ZZ"])
beta = 1.0
ideal_state = simple_purify_hamiltonian(H*beta)
relative_beta_error = np.linspace(0.9,1.1,11)
faulty_states = [simple_purify_hamiltonian(H*beta*r) for r in relative_beta_error]
fidelities = [state_fidelity(ideal_state,f) for f in faulty_states]
df = {"Relative Beta":relative_beta_error, "Fidelity":fidelities}
px.scatter(df,x="Relative Beta",y="Fidelity")

Now let's analyze how much of the error is usually associated to the temperature of the Hamiltonian and how much outside of it.

In [3]:
result = get_results("../scripts_lap/job_13_qasm")[0]
result = result.change_locality(2)
timestep=-1
Ho = np.real(result.coriginal*result.betas[timestep])
Hp = np.real(result.cfaulties[timestep])
DeltaH = Hp-Ho
projection = Ho * np.dot(Ho,DeltaH)/(np.linalg.norm(Ho)**2)
orthogonal = DeltaH - projection
assert np.abs(np.dot(orthogonal,Ho)) < 1e-8, f"The projection is {np.abs(np.dot(orthogonal,DeltaH))}"
print(f"Projecting the preparation hamiltonian, we get that the component parallel to Ho is {np.linalg.norm(projection)} and the orthogonal one is {np.linalg.norm(orthogonal)}")
print(f"The temperature of the original hamiltonian is {np.linalg.norm(Ho)} and that of the preparation is{np.linalg.norm(Hp)}")
px.bar(data_frame={"Ho":Ho,"projection":projection,"orthogonal":orthogonal},barmode="group",y=["Ho","projection","orthogonal"]).show()

Projecting the preparation hamiltonian, we get that the component parallel to Ho is 0.20533220323738646 and the orthogonal one is 0.28733686887396837
The temperature of the original hamiltonian is 1.2087952279950032 and that of the preparation is1.0437914151160665


An other important matter is whether our ideal state leads to a degenerate kernel of the constraint matrix.

In [64]:
from gibbs.learning.constraint_matrix import ConstraintMatrixFactory

H:SparsePauliOp = (lattice_hamiltonian(6,1/4,-1,["X"],["XX","YY","ZZ"],periodic=False)+ 1e-9*noise_hamiltonian(6)).simplify()
num_qubits=H.num_qubits
cmat = ConstraintMatrixFactory(num_qubits,2,3)
K,E = cmat.create_cmat(state= simple_purify_hamiltonian(H),shots = None)
co = cmat.learning_basis.pauli_to_vector(H)

# num_qubits=result.num_qubits
# cmat = ConstraintMatrixFactory(num_qubits,2,3)
# # K,E = cmat.create_cmat(state= simple_purify_hamiltonian((Hp[:cmat.learning_basis.size],cmat.learning_basis)),shots = 1e5)
# K,E = cmat.create_cmat(state = result.state_ansatz(timestep),shots =1e6)
# co = result.coriginal*result.betas[timestep]

In [65]:
print(K.shape)
s,v = spectral_dec(K)
df = {"Ordered Singular Values":np.arange(len(s),dtype=int),"Singular Value":np.array(s)}
fig = px.line(df,x="Ordered Singular Values",y="Singular Value",log_y=True,markers=".")
fig.update_xaxes(visible=False)
fig.update_yaxes(exponentformat="power",gridcolor="gray")
fig.update_layout(width=500,height=400,plot_bgcolor="white",)
fig.show()
# fig.write_image("../figures/degeneratekernel.pdf")

(207, 63)


In [ ]:
cand = candidateV2(v[-1],co[:result.local_size(1)])
# cand = candidate(v[-1],co)
candB = candidateV3(v[-1],co[:result.local_size(1)])

# candB = candidateV2(v[-2],co[:result.local_size(1)])
# state = simple_purify_hamiltonian((co,cmat.learning_basis))
state = result.state_vector(timestep)
stateA = simple_purify_hamiltonian((cand,cmat.learning_basis))
stateB = simple_purify_hamiltonian((candB,cmat.learning_basis))

print(f"The fidelity with the first and second reconstruction is {state_fidelity(state,stateA)} and {state_fidelity(state,stateB)}")
df = {"Original":np.real_if_close(co),"Candidate":np.real_if_close(cand),"Preparation":np.real_if_close(result.cfaulties[timestep]),"Candidate2":candB}
px.bar(df,y=["Original","Candidate","Preparation","Candidate2"],barmode="overlay")

The fidelity with the first and second reconstruction is 0.9952898956844543 and 0.9924351079733712


In [ ]:
Ho = lattice_hamiltonian(6,1/4,-1,["Z"],["XX","YY","ZZ"])
H_noise = noise_hamiltonian(Ho.num_qubits)
def infidelity_shots(shots,noise,Ho,Hnoise):
    num_qubits=Ho.num_qubits
    H = (Ho + noise*Hnoise).simplify()
    cmat = ConstraintMatrixFactory(num_qubits,2,3)
    co = cmat.learning_basis.pauli_to_vector(Ho)
    K,E = cmat.create_cmat(state= simple_purify_hamiltonian(H),shots = shots)
    s,v = spectral_dec(K)
    cand = candidateV2(v[:,-1],co[:KLocalPauliBasis(1,num_qubits=).size])
    state_ideal = simple_purify_hamiltonian(H)
    state_learned = simple_purify_hamiltonian((cand,cmat.learning_basis))
    return 1-state_fidelity(state_ideal,state_learned)

SyntaxError: invalid syntax (1296098151.py, line 10)

In [ ]:
from tqdm import tqdm
shots = np.logspace(4,8,50)
fidelities = [infidelity_shots(s,0.05,Ho,H_noise) for s in tqdm(shots)]

100%|██████████| 50/50 [03:15<00:00,  3.92s/it]


In [ ]:
df = {"Shots":shots,"Fidelities":fidelities,"Prediction":shots**(-0.5)}
fig = px.scatter(df,x="Shots",y=["Fidelities","Prediction"],log_x=True,log_y=True)
fig.show()

In [ ]:
N=50
print(f"{KLocalPauliBasis(2,N).size*KLocalPauliBasis(3,N).size:.0e},{KLocalPauliBasis(4,N).size:.0e}")
print(f"{4**N:.0e}")

1e+06,9e+03
1e+30


In [ ]:
lens = [KLocalPauliBasis(4,i).size for i in range(50)]
px.scatter(lens)